In [1]:
import numpy as np
from load import load_data
from convert import convert
#from pmc import *
#from brute_force import planes_def
from pmc_4p import *
from brute_force_4pPMC import planes_def

In [2]:
#%matplotlib widget
#%matplotlib inline
%matplotlib qt
#%matplotlib gtk
import matplotlib.pyplot as plt
import matplotlib.font_manager
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
plt.rcParams.update({
    'pgf.rcfonts': False,
})
plt.rcParams['font.family'] = 'serif'
#from matplotlib.backends.backend_pgf import FigureCanvasPgf
#matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)

In [3]:
alf_inter = 1
alf_plane = 0.5

# Input

In [4]:
data = load_data(13)
d = convert(data)
P1, P2, SP1, SP2 = planes_def(data,d)
print(SP1,SP2)
#print(P1.coeff)
#print(P2.coeff)

## stress state computation at failure
#sig3 = 10
#sig2 = 60
#sig1 = (1/P2.A)*(-P2.B*sig2-P2.C*sig3+1)
#print(sig1)


[103.81364665  13.07220553  13.07220553]
[11.19819018 34.24979471 34.24979471]
[[ 71.5   10.    10.    30.5   61.5    0.  ]
 [ 91.08  20.    20.    44.72  71.08   0.  ]
 [ 95.9   20.    20.    45.3   75.9    0.  ]
 [ 98.4   20.    20.    46.1   78.4    0.  ]
 [ 99.98  20.    20.    46.65  62.28   0.  ]
 [110.9   30.    30.    57.    80.9    0.  ]
 [114.5   30.    30.    58.2   84.5    0.  ]
 [127.3   40.    40.    65.73  87.3    0.  ]
 [125.5   40.    40.    68.5   85.5    0.  ]
 [129.4   40.    40.    69.8   89.4    0.  ]
 [138.1   50.    50.    79.4   88.1    0.  ]
 [142.1   50.    50.    80.7   92.1    0.  ]
 [151.1   60.    60.    88.12  91.1    0.  ]
 [150.8   60.    60.    90.3   90.8    0.  ]
 [153.8   60.    60.    91.3   93.8    0.  ]
 [ 88.14  46.85  10.    48.33  55.28  28.12]
 [ 70.6   49.4   15.    45.    48.7   37.8 ]
 [ 83.9   62.1   22.    56.    54.4   39.7 ]
 [ 77.5   70.5   20.    56.    54.3   53.5 ]]
[[24.9   0.    0.    8.3  24.9   0.  ]
 [29.7   0.    0.    9.9 

In [5]:
eq_pts_transisiton_P12 = np.zeros((6,3,3))
for i in range(6):
    eq_pts_transisiton_P12[i] = np.array([get_plane_normal_6_cycle(P1,i),get_plane_normal_6_cycle(P1,i-1),get_plane_normal_6_cycle(P2,i)])

ones = np.ones((6,3))
pts_transisiton_P12 = np.linalg.solve(eq_pts_transisiton_P12, ones).transpose()

In [6]:
data = data[0,:]
print(data[:3])

[29.7  0.   0. ]


In [7]:
eq_pts_P2 = np.array([get_plane_normal_6_cycle(P2,0),get_plane_normal_6_cycle(P2,1),get_plane_normal_6_cycle(P2,2)])
pts_P2 = np.linalg.solve(eq_pts_P2,np.ones((3,1)))

In [8]:
p_trans = (pts_transisiton_P12[0,:]+pts_transisiton_P12[1,:]+pts_transisiton_P12[2,:])/3

In [9]:
sig_pts = np.concatenate((P1.pts,P2.pts),axis=1) #miss pts_P2
pts_with_largest_coord = sig_pts[:,np.argmax(np.max(sig_pts, axis=0))]
dist = np.dot(np.ones(3)/np.sqrt(3),pts_with_largest_coord)*1.1

In [10]:
pts_P1_far = p_plane_intersection_6(P1,dist)

In [11]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
#ax.plot(*P1.pts,'k.')
#ax.plot(*P2.pts,'k.')
#ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
ax.plot(*pts_P2,'ko',alpha=0)
ax.plot(*pts_transisiton_P12,'ro',alpha=alf_inter)
ax.plot(*pts_P1_far,'bo',alpha=alf_inter)
ax.set_xlabel('[MPa]')
ax.set_ylabel('[MPa]')
ax.set_zlabel('[MPa]')
def get_verts(coord):
    return [list(zip(*cord))]

## Draw P2 plane

In [12]:
for i in range(6):
    cord = np.zeros((3,3))
    cord[:,0] = pts_P2[:,0]
    cord[:,1] = pts_transisiton_P12[:,i]
    cord[:,2] = pts_transisiton_P12[:,(i+1)%6]
    
    verts = get_verts(cord)
    poly = Poly3DCollection(verts,linewidths=1, alpha=alf_plane)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)

## Draw P1 plane

In [13]:
for i in range(6):
    cord = np.zeros((3,4))
    cord[:,0] = pts_transisiton_P12[:,i]
    cord[:,1] = pts_transisiton_P12[:,(i+1)%6]
    cord[:,2] = pts_P1_far[:,(i+1)%6]
    cord[:,3] = pts_P1_far[:,i%6]
    
    verts = get_verts(cord)
    poly = Poly3DCollection(verts,linewidths=1, alpha=alf_plane)
    poly.set_facecolor('b')
    ax.add_collection3d(poly)

In [14]:
ax.grid(b=None)
ax.set_zticks([0,40,80,120,160])
ax.set_xticks([0,40,80,120,160])
ax.set_yticks([0,40,80,120,160])
fig.show()
#fig.savefig('3D.pdf')

# 2D view

In [15]:
new_base = np.zeros((3,3))

new_z = np.array([1,1,1])
new_y = np.array([-1,-1,2])
new_x = np.array([-1,1,0])
new_base[:,0] = new_x /np.linalg.norm(new_x )
new_base[:,1] = new_y /np.linalg.norm(new_y )
new_base[:,2] = new_z /np.linalg.norm(new_z )
new_base_transform = np.linalg.inv(new_base)

In [16]:
#new_sig = new_base_transform @ sig_pts
new_transi = new_base_transform @ pts_transisiton_P12
new_P2 = new_base_transform @ pts_P2
new_far_P1 = new_base_transform @ pts_P1_far

In [17]:
transi_dist_min = np.min(new_transi[2,:])
transi_dist_max = np.max(new_transi[2,:])
transi_dist_mid = (transi_dist_min+transi_dist_max)/2
near_inter = p_plane_intersection_6(P2,transi_dist_min*0.4)
near_transP2 = p_plane_intersection_6(P2,transi_dist_mid*1)
near_transP1 = p_plane_intersection_6(P1,transi_dist_mid*1)
new_near = new_base_transform @ near_inter
new_transP2 = new_base_transform @ near_transP2
new_transP1 = new_base_transform @ near_transP1

In [18]:
transi_dist_max = np.max(new_transi[2,:])
t = 0.5
dist = transi_dist_max*t + transi_dist_min*(1-t)
#offset = 2 if np.allclose(new_transi[0,0],transi_dist_max) else 0
offset = 2 if p_trans[0]>p_trans[1] else 0
middle_inter = p_plane_intersection_12(P1,P2,dist,offset)
new_middle = new_base_transform @ middle_inter

In [19]:
o_sig = d['o'][:,:3].transpose()
o_pts = np.zeros(o_sig.shape)
o_pts[0,:] = o_sig[1,:]
o_pts[1,:] = o_sig[2,:]
o_pts[2,:] = o_sig[0,:]
new_o_pts = new_base_transform @ o_pts

In [20]:
p = np.unique(d['o'][:,3])
pts_tt1p = p_planes(p[0],P1,P2,p_trans,offset)
pts_tt2p = p_planes(p[1],P1,P2,p_trans,offset)
pts_tt3p = p_planes(p[3],P1,P2,p_trans,offset)
pts_tt1 = p_planes(p[2],P1,P2,p_trans,offset)

In [21]:
new_tt1p = new_base_transform @ pts_tt1p
new_tt2p = new_base_transform @ pts_tt2p
new_tt3p = new_base_transform @ pts_tt3p
new_tt1 = new_base_transform @ pts_tt1

In [24]:
plt.close()
#plt.axis('equal')
p1, = plt.plot(new_o_pts[0,:3],new_o_pts[1,:3],'r*', label='p={} MPa'.format(p[0]))
p2, = plt.plot(new_o_pts[0,3],new_o_pts[1,3],'b.',label='p={} MPa'.format(p[1]))
p3, = plt.plot(new_o_pts[0,4:6],new_o_pts[1,4:6],'gp',label='p={} MPa'.format(p[3]))
p4, = plt.plot(new_o_pts[0,6],new_o_pts[1,6],'yX',label='p={} MPa'.format(p[2]))

#p5, = plt.fill(new_middle[0,:],new_middle[1,:],edgecolor='k',fill=False,label='12 sided - P2 and P1')
plt.fill(new_tt1p[0,:],new_tt1p[1,:],edgecolor='r',fill=False)
plt.fill(new_tt2p[0,:],new_tt2p[1,:],edgecolor='b',fill=False)
plt.fill(new_tt3p[0,:],new_tt3p[1,:],edgecolor='g',fill=False)
plt.fill(new_tt1[0,:],new_tt1[1,:],edgecolor='y',fill=False)
plt.legend(handles=[p1, p2, p3, p4])
x = np.linspace(0,70)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')

x = np.linspace(-90,0)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')

plt.xlabel('[MPa]')
plt.ylabel('[MPa]')
plt.axis('equal')
#plt.axis('off')
plt.show()

In [23]:
plt.close()
p1, = plt.fill(new_far_P1[0,:],new_far_P1[1,:],edgecolor='b',fill=False,label='6 sided - P1')
p2, = plt.fill(new_near[0,:],new_near[1,:],edgecolor='r',fill=False,label='6 sided - P2')
plt.fill(new_transP2[0,:],new_transP2[1,:],edgecolor='r',fill=False)
plt.fill(new_transP1[0,:],new_transP1[1,:],edgecolor='b',fill=False)
p3, = plt.fill(new_middle[0,:],new_middle[1,:],edgecolor='k',fill=False,label='12 sided - P2 and P1')
plt.legend(handles=[p1, p2, p3])

x = np.linspace(0,100)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')

x = np.linspace(-110,0)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')

plt.xlabel('[MPa]')
plt.ylabel('[MPa]')
plt.axis('equal')
#plt.axis('off')
plt.show()

In [48]:
plt.close()